# Demo - Model with Multiple Outputs

In [1]:
import torch
import torch.nn as nn

import torchattacks
from torchattacks import PGD, FGSM

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Sample image and label
image = torch.randn((1, 100)).cuda()
label = torch.tensor([1]).cuda()

### AttributeError: 'tuple' object has no attribute 'log_softmax'

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Linear(100,10)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(10,10)      
        
    def forward(self,x):
        feature = self.relu(self.layer1(x))
        out = self.layer2(feature)

        return out, feature

In [5]:
model = CNN().cuda()

In [6]:
atk = PGD(model, eps=0.3, alpha=0.1, steps=7)

In [7]:
adv_image = atk(image, label)

AttributeError: 'tuple' object has no attribute 'log_softmax'

### To resolve the issue, please add a module that outputs only logits.

In [8]:
class SelectOutput(nn.Module):
    def __init__(self):
        super(SelectOutput, self).__init__()
        
    def forward(self,x):
        out = x[0]
        return out

In [9]:
new_model = nn.Sequential(
    model,
    SelectOutput()
)

In [10]:
atk = PGD(new_model, eps=0.3, alpha=0.1, steps=7)

In [11]:
# No Error
adv_image = atk(image, label)